# 1.0 – ETAPA UM – ENTENDENDO O NEGÓCIO

A PProductions é um estúdio de cinema que atua no mercado global de entretenimento, produzindo longas-metragens para cinema e streaming. Sua receita é gerada principalmente por bilheteria, distribuição em plataformas digitais, licenciamento e merchandising. O público-alvo é amplo, abrangendo diferentes faixas etárias e preferências culturais.

O setor é altamente competitivo, com grandes estúdios e plataformas de streaming disputando a atenção do público. Os desafios incluem elevados custos de produção e marketing, forte concorrência internacional e mudanças rápidas no comportamento dos consumidores, cada vez mais orientados para o streaming. Em contrapartida, existem oportunidades ligadas ao uso de dados para reduzir incertezas, apoiar decisões de investimento e identificar fatores de sucesso em futuros lançamentos.

A estrutura organizacional envolve áreas criativas, produção, marketing/distribuição e finanças. Decisores estratégicos precisam equilibrar risco e retorno, apoiando-se em análises quantitativas para melhorar a taxa de acerto em decisões de portfólio. O maior risco é a incerteza de audiência, que pode comprometer grandes investimentos, enquanto a principal oportunidade é aumentar a previsibilidade por meio de modelos de ciência de dados.

# 2.0 – ETAPA DOIS – PROBLEMA DE NEGÓCIO

O estúdio de cinema PProductions, em Hollywood, enfrenta um desafio estratégico: decidir qual será o próximo filme a ser produzido em um mercado de altíssimo investimento e intensa competição. Uma decisão equivocada pode representar perdas de milhões de dólares, enquanto uma escolha bem-informada pode gerar retornos expressivos e fortalecer a posição do estúdio no mercado global.

O time executivo precisa de análises confiáveis que reduzam a incerteza e apoiem decisões críticas sobre portfólio de produção. Para isso, é necessário compreender quais fatores mais impactam o sucesso de um filme, prever indicadores de audiência como a nota do IMDb, avaliar o potencial de faturamento com base nas características do filme e explorar novas formas de análise, como o uso de NLP para inferir gêneros a partir de sinopses.

O objetivo deste projeto é traduzir dados históricos do setor cinematográfico em insights e modelos preditivos que apoiem decisões de alto risco financeiro. A solução deve combinar análise exploratória, modelagem estatística e protótipos de machine learning, sempre com foco em clareza, aplicabilidade e valor de negócio.

# 3.0 – ETAPA TRÊS – COLETA DE DADOS

## 3.1. - Imports:
Nesta seção, todas as bibliotecas e módulos necessários são importados. Essas bibliotecas fornecem a funcionalidade exigida para diversas tarefas de manipulação de dados, visualização e modelagem.

In [24]:
# Manipulação e análise de dados
import math
import numpy as np
import pandas as pd
from pathlib import Path

# Visualização de dados
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from tabulate import tabulate

# Utilidades para notebooks
from IPython.display import Image
from IPython.core.display import HTML

# Manipulação de datas
import datetime


## 3.2. - Funções Auxiliares:
Esta seção contém várias funções auxiliares que serão usadas ao longo do projeto para realizar tarefas como pré-processamento de dados, cálculo de erros e validação cruzada. Essas funções ajudam a melhorar a organização e a reutilização do código.

## 3.3. - Carregando os Dados: